In [ ]:
import rdflib
import grizzly as gz
from grizzly.types import LangStr, TypedStr
import time

In [ ]:
graph = rdflib.Graph()
g = graph.parse('nalt-core_dwn_20240716.ttl', format='ttl')

In [ ]:
g.serialize(destination='nalt-core_dwn_20240716.nt', format='nt')

In [ ]:
import pyximport; pyximport.install()

import cyparser
import cyparser2
import cyparser3
import cyparser4
import cyparser5
import cyparser6
import cyparser7


In [ ]:
def parse(string):

    # t0 = time.time()

    whitespace = {' ', '\n', '\t'}

    triples = []

    subject = None
    predicate = None
    object = None

    start = 0
    uri = False
    literal = False
    lang = False

    for i, c in enumerate(string):
        if uri and c != '>':
            continue
        elif literal and c != '"':
            continue
        elif c == '<' and not literal:
            start = i + 1
            uri = True
        elif c == '>' and uri:
            value = string[start:i]
            start = 0
            uri = False
            if subject is None:
                subject = value
            elif predicate is None:
                predicate = value
            else:
                triples.append((subject, predicate, value))
                subject = None
                predicate = None
        elif c == '"' and not literal:
            start = i + 1
            literal = True
        elif c == '"' and literal:
            object = string[start:i]
            start = 0
            literal = False
            # if not (subject and predicate):
            #     raise Exception
        elif c == '@' and object:
            start = i + 1
            lang = True
        # elif c =='^' and object:

        elif c in whitespace and lang:
            value = string[start:i]
            start = 0
            lang = False
            triples.append((subject, predicate, LangStr(object).set_lang(value)))
            subject = None
            predicate = None
            object = None
        elif c in whitespace and object:
            triples.append((subject, predicate, TypedStr(object)))
            subject = None
            predicate = None
            object = None

    # print(time.time() - t0)
    return triples

In [ ]:
def load_triples_grizzly(filename):

    with open(filename) as f:
        string = f.read()
    triples = parse(string)
    return gz.Graph(triples)

def load_triples_rdflib(filename):

    graph = rdflib.Graph()
    g = graph.parse(filename, format='nt')
    return graph


In [ ]:
def load_triples_grizzly_cython(filename, parser):

    with open(filename) as f:
        string = f.read()
    triples = parser.parse(string)
    return gz.Graph(triples)

In [ ]:
filename = 'nalt-core_dwn_20240716.nt'
with open(filename) as f:
    string = f.read()

In [ ]:
%timeit -n 1 -r 1 load_triples_rdflib(filename)

In [ ]:
%timeit -n 1 -r 1  load_triples_grizzly(filename)

In [ ]:
%timeit -n 1 -r 1 cyparser2.load_triples_grizzly_cython(filename)

In [ ]:
%timeit -n 1 -r 1 parse(string)

In [ ]:
%timeit cyparser.parse(string)


In [ ]:
%timeit cyparser2.parse(string)

In [ ]:
%timeit cyparser3.parse(string)

In [ ]:
%timeit cyparser4.parse(string)

In [ ]:
%timeit cyparser5.parse(string)

In [ ]:
%timeit cyparser6.parse(string.encode('utf-8'))

In [ ]:
%timeit cyparser7.parse(string.encode('utf-8'))

In [ ]:
t = cyparser7.parse(string.encode('utf-8'))
t

In [ ]:
g = gz.Graph(t)

In [ ]:
len(t)

In [ ]:
rdflib_g = load_triples_rdflib(filename)
len(rdflib_g)

In [ ]:
g2 = gz.read_turtle(filename)
g2

In [ ]:
g2.query('''SELECT ?s ?o WHERE {?s a ?o .} LIMIT 10''').decode()

In [ ]:
t = cyparser7.parse("""
<bla> <bli> <blup>, <x>, "test" ;
     <a> <b> .
""".encode('utf-8'))
t